In [0]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

In [2]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE

def read_tfrecord(example):
    features = {
        "image": tf.FixedLenFeature((), tf.string), # tf.string means byte string
        "label": tf.FixedLenFeature((), tf.string),
        "one_hot_label": tf.FixedLenFeature((), tf.string)
    }
    example = tf.parse_single_example(example, features)
    image = tf.image.decode_jpeg(example['image'])
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    one_hot_label = tf.io.decode_raw(example['one_hot_label'], out_type=tf.uint8) # 'decode' byte string into byte list
    one_hot_label = tf.cast(one_hot_label, tf.float32)  # convert one hot labels to floats
    one_hot_label = tf.reshape(one_hot_label, [5])  # explicit fixed size needed on TPU
    label = example['label']  # byte string
    return image, label, one_hot_label
  
def load_dataset(filenames):  
  # read from tfrecs
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=32)  # this will read from multiple GCS files in parallel
  dataset = records.map(read_tfrecord, num_parallel_calls=32)
  return dataset

def features_and_targets(image, label, one_hot_label):
  feature = image
  target = one_hot_label
  return feature, target  # for training, a Keras model needs 2 items: features and targets

def get_batched_dataset(filenames):
  dataset = load_dataset(filenames)
  dataset = dataset.map(features_and_targets, num_parallel_calls=32)
  dataset = dataset.cache()  # This dataset fits in RAM
  dataset = dataset.repeat()
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # drop_remainder needed on TPU
  dataset = dataset.prefetch(-1) # prefetch next batch while training  (-1: autotune prefetch buffer size)
  # should shuffle too but this dataset was well shuffled on disk already
  return dataset

def get_training_dataset():
  return get_batched_dataset(training_filenames)

def get_validation_dataset():
  return get_batched_dataset(validation_filenames)

conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files
Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 5s 0us/step
Instructions for updating:
Use tf.cast instead.
Epoch 1/8
93/93 [==============================] - 40s 430ms/step - loss: 0.8366 - acc: 0.6939 - val_loss: 0.6061 - val_acc: 0.7857
Epoch 2/8
93/93 [==============================] - 25s 271ms/step - loss: 0.3856 - acc: 0.8716 - val_loss: 0.6015 - val_acc: 0.7798
Epoch 3/8
93/93 [==============================] - 25s 272ms/step - loss: 0.2282 - acc: 0.9335 - val_loss: 0.7057 - val_acc: 0.7634
Epoch 4/8
93/93 [==============================] - 25s 271ms/step - loss: 0.1781 - acc: 0.9442 - val_loss: 0.8777 - val_acc: 0.7455
Epoch 5/8
93/93 [==============================] - 25s 272ms/step - loss: 0.2210 - acc: 0.9177 - val_loss: 0.7471 - val_acc: 0.7693
Epoch 6/8
93/93 [==============================] - 25s 271ms

In [0]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE

conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files
Epoch 1/8
93/93 [==============================] - 29s 312ms/step - loss: 0.8949 - acc: 0.6650 - val_loss: 0.6044 - val_acc: 0.7887
Epoch 2/8
93/93 [==============================] - 25s 274ms/step - loss: 0.4172 - acc: 0.8555 - val_loss: 0.6085 - val_acc: 0.7812
Epoch 3/8
93/93 [==============================] - 25s 272ms/step - loss: 0.2793 - acc: 0.9113 - val_loss: 0.8805 - val_acc: 0.6994
Epoch 4/8
93/93 [==============================] - 25s 273ms/step - loss: 0.2589 - acc: 0.9073 - val_loss: 0.6945 - val_acc: 0.7723
Epoch 5/8
93/93 [==============================] - 25s 273ms/step - loss: 0.1815 - acc: 0.9395 - val_loss: 0.5181 - val_acc: 0.8318
Epoch 6/8
93/93 [==============================] - 25s 273ms/step - loss: 0.0911 - acc: 0.9724 - val_loss: 0.5432 - val_acc: 0.8289
Epoch 7/8
93/93 [==============================] - 25s 273ms/step - loss: 0.0490 - acc: 0.9913 - val_loss: 0.5538

In [3]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE


# conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(192, 192, 3))
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 9s 0us/step
Epoch 1/8
93/93 [==============================] - 32s 342ms/step - loss: 7.5292 - acc: 0.5128 - val_loss: 12.2565 - val_acc: 0.2396
Epoch 2/8
93/93 [==============================] - 23s 244ms/step - loss: 7.1831 - acc: 0.5484 - val_loss: 12.2593 - val_acc: 0.2366
Epoch 3/8
93/93 [==============================] - 23s 244ms/step - loss: 6.1310 - acc: 0.6095 - val_loss: 12.3044 - val_acc: 0.2366
Epoch 4/8
93/93 [==============================] - 23s 245ms/step - loss: 5.3413 - acc: 0.6610 - val_loss: 12.2805 - val_acc: 0.2381
Epoch 5/8
93/93 [==============================] - 23s 245ms/step - loss: 5.0811 - acc: 0.6798 - val_loss: 11.5826 - val_acc: 0.2723
Epoch 6/8
93/93 [==============================] - 23s 246ms/step - loss: 4.7639 - acc: 0.6989 - val_loss: 9.2631 - val_acc: 0.4122
Epoch 7/8
93/93 [==============================] - 23s 246ms/step - loss: 3.6250 - acc: 0.7658 - val_loss: 7.6745 - val_acc: 0.5134
Epoch 